In [33]:
import numpy as np
import pandas as pd
import time 
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

## 用户信息表：

In [35]:
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/train.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/test.csv",encoding="gb2312") # 注意自己数据路径
test=test.fillna(-1)

dataset= pd.concat([train,test],axis=0).reset_index()
label=dataset[['用户标识','标签']]
dataset=dataset.drop(['index','用户标识'],axis=1)

dataset=pd.get_dummies(dataset,columns=dataset[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)
dataset #23899 rows × 24 columns
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
dataset=poly.fit_transform(dataset)
#dataset.shape
df = pd.DataFrame(dataset.reshape(23899,301))
df.interpolate(axis=1).values.reshape(dataset.shape)

#dataset
#trains=pd.merge(label,df,left_index=True,right_index=True,how='outer')
trains=pd.concat([label,df],axis=1)

In [36]:
#trains
trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/featureall_ui.csv",index=None,encoding="gb2312") #23899 rows × 1171 columns 没有删去用户标识

## 用户浏览行为训练表：

In [37]:
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/user_browse_test.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/user_browse_train.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)

temp=dataset[:]
temp.drop('放款时间',axis=1, inplace=True)
print(temp.shape)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户浏览行为缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
temp.fillna(0,inplace=True) #0填充缺失值
print(temp.shape)
trains=pd.merge(trains,temp,how='left', on = "用户标识")
print(trains.shape)

(23899, 39)
(23899, 40)
(23899, 342)


d:\code environment\python\python3.6\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [38]:
#trains
trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/featureall_browse.csv",index=None,encoding="gb2312") #0填充 23899 rows × 1171 columns 没有删去用户标识

## 用户银行流水记录表：

In [39]:
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bank_train.csv",encoding="gb2312")
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bank_test.csv",encoding="gb2312")
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
temp.drop('放款时间',axis=1, inplace=True)
print(temp.shape)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
temp.fillna(0,inplace=True)
print(temp.shape)
trains=pd.merge(trains,temp,how='left', on = "用户标识")
print(trains.shape)

(23899, 25)
(23899, 26)
(23899, 367)


d:\code environment\python\python3.6\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [40]:
#trains
trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/featureall_bank.csv",index=None,encoding="gb2312") #0填充 23899 rows × 1171 columns 没有删去用户标识

## 用户账单表初级特征：

In [41]:
beifen=trains
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_prifeature_train.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_prifeature_test.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
temp.drop('放款时间',axis=1, inplace=True)
print(temp.shape)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户账单表初级特征缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 
for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
#emp.fillna(temp.mean(),inplace=True)
temp.fillna(0,inplace=True)
print(temp.shape)
trains=pd.merge(trains,temp,how='left', on = "用户标识")
print(trains.shape)#23899 rows × 445  columns

(23899, 56)
(23899, 79)
(23899, 445)


d:\code environment\python\python3.6\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/featureall_billpri.csv",index=None,encoding="gb2312") #0填充 23899 rows × 1171 columns 没有删去用户标识

## 用户账单表特征_时间已知：

In [43]:
beifen=trains[:]
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_feature_t_train.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_feature_t_test.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
temp.drop('放款时间',axis=1, inplace=True)
print(temp.shape)#(23899, 473)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户账单表特征_时间已知缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 

for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
temp.fillna(0,inplace=True)
print(temp.shape)#(23899, 706)
trains=pd.merge(trains,temp,how='left', on = "用户标识")
print(trains.shape)#23899 rows × 1150 columns

d:\code environment\python\python3.6\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(23899, 473)
(23899, 706)
(23899, 1150)


## 用户账单表特征_时间未知

In [44]:
beifen=trains[:]
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_feature_nt_train.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_featyre_nt_test.csv",encoding="gb2312") # 注意自己数据路径

#2017.02.04随机删除部分时间未知样本，保证训练集与测试集的特征分布相似
print(train.shape)
train已知=train[train['时间未知上期账单金额sum'].isnull().values==True]#(4988, 241)
print(train已知.shape)
train未知=train[train['时间未知上期账单金额sum'].isnull().values==False]#(5012, 241)
print(train未知.shape)
t1,t2=train_test_split(train未知, test_size = 0.5,random_state=36)#
print(t1.shape)
train=pd.concat([train已知,t1],axis=0)
print(train.shape)

####################################################################

dataset= pd.concat([train,test],axis=0)
#dataset['账单时间未知标志']=1
temp=dataset[:]
#temp.drop('放款时间',axis=1, inplace=True)
print(temp.shape)#(21393, 242)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户账单表特征_时间未知缺失统计']=(dataset==-1).sum(axis=1)        
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
temp.fillna(0,inplace=True)
print(temp.shape)#(21393, 243)
trains=pd.merge(trains,temp,how='inner', on = "用户标识")#1154+243
print(trains.shape)#(21393, 1392)

(10000, 241)
(4988, 241)
(5012, 241)
(2506, 241)
(7494, 241)
(21393, 241)
(21393, 242)
(21393, 1391)


In [45]:
train未知.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_without_time.csv",index=None,encoding="gb2312")
trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/feature_bill_dropntp.csv",index=None,encoding="gb2312") #0填充 (61612, 1391) 删除了部分账单时间未知样本

## 用户账单表特征_整体

In [46]:
beifen=trains[:]
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_feature_al_train.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_feature_al_test.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
#temp.drop('放款时间',axis=1, inplace=True)
print(temp.shape)#(69495, 242)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户账单表特征_时间未知缺失统计']=(dataset==-1).sum(axis=1)        
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
temp.fillna(0,inplace=True)
print(temp.shape)#(69495, 243)
trains=pd.merge(trains,temp,how='left', on = "用户标识")#1154+243
print(trains.shape)#(69495, 1633)

(23899, 241)
(23899, 242)
(21393, 1632)


In [47]:
#trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/featureall_bill_total.csv",index=None,encoding="gb2312") #0填充 69495 rows × 608columns 只取整体特征
trains

,用户标识,标签,0,1,2,3,4,5,6,7,...,去重后整体本期账单金额var,去重后整体调整金额var,去重后整体循环利息var,去重后整体可用余额var,去重后整体预借现金额度var,去重后整体上期还款金额var与整体上期账单金额var差值,去重后整体信用卡额度var与整体本期账单余额var差值,去重后整体可用余额var与整体预借现金额度var差值,去重后整体本期账单最低还款额var与整体循环利息var之和,用户账单表特征_时间未知缺失统计_y
0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240
1,4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,78.000994,0.000000,0.000000,18.103521,87.803786,-20.167102,62.381889,-69.700264,0.954005,0
2,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15
3,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,97.932595,0.000000,0.000000,67.875596,78.664177,-198.968749,-232.767048,-10.788580,56.628128,0
4,8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
5,9,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,111.034824,0.000000,0.000000,0.000000,99.776144,-155.005745,-65.653716,-99.776144,21.106390,31
6,11,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15
7,14,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,76.159948,0.000000,16.177671,0.000000,42.433718,-14.720737,84.248481,-42.433718,18.376104,31
8,15,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.266792,0.000000,0.000000,0.000000,0.000000,-364.006189,-368.039519,0.000000,95.232148,0
9,16,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,112.144517,28.257646,0.000000,0.000000,0.000000,-18.381554,-111.829754,0.000000,91.857791,0


# 特征汇总

# 测评

In [16]:
dataset= pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/feature_bill_dropntp.csv",encoding="gb2312") # 注意自己数据路径
#dataset=trains[:]
beifen=0
trains=0

# dataset.drop('放款时间_x',axis=1, inplace=True)
# dataset.drop('放款时间_y',axis=1, inplace=True)
# dataset.drop('放款时间_x.1',axis=1, inplace=True)
# dataset.drop('放款时间_y.1',axis=1, inplace=True)

from sklearn.preprocessing import MinMaxScaler
#dataset=trains[:]
#检测重复特征名称,并删除
names=dataset.columns.tolist()
print(len(names))
print(len(set(names)))
print("dataset大小：")
print(dataset.shape)
myset = set(names)
for item in myset:
    #dataset[item] = MinMaxScaler().fit_transform(dataset[item])
    if names.count(item)>1:
        print(item)
        dataset=dataset.drop(item,axis=1)
print("新dataset大小：")
print(dataset.shape)

1391
1391
dataset大小：
(21393, 1391)
新dataset大小：
(21393, 1391)


In [17]:
#trains

In [18]:
tests=dataset[(dataset['标签']<0)]
trains=dataset[(dataset['标签']>=0)]
时间未知= pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_without_time.csv",encoding="gb2312")[['用户标识','时间未知上期账单金额sum']] # 注意自己数据路径
t1,t2=train_test_split(时间未知, test_size = 0.5,random_state=36)#
trains=pd.merge(trains,t1,how='left', on = "用户标识")


In [19]:
trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/检查一下.csv",index=None,encoding="gb2312")

In [20]:
#trains=trains[trains['时间未知上期账单金额sum_x'].isnull().values==True] #(39830, 241)

In [21]:
#trains.drop('时间未知上期账单金额sum_x',axis=1, inplace=True)
#trains.shape
#sns.FacetGrid(dataset, hue="标签", size=5).map(plt.scatter, "账单时间未知标志", "标签").add_legend()

## KS测评：

In [22]:
#t1=trains.columns
#t2=tests.columns
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

In [23]:

#trains=trains.drop(['放款时间'],axis=1) #55596行  
#tests=tests.drop(['放款时间'],axis=1)#测试集:13899行
online_test=tests
#train_xy,offline_test = train_test_split(trains, test_size = 0.25,random_state=36)#41697/13899
#train,val = train_test_split(train_xy, test_size = 0.333,random_state=24)#27811/13886
#train,val = train_test_split(trains, test_size = 0.25,random_state=20)#41697/13899
#train_xy,offline_test = train_test_split(trains, test_size = 0.25,random_state=10)#41697/13899

#2017.02.04  
train,val = train_test_split(trains, test_size = 0.25,random_state=36)#41697/13886
offline_test=val[:]
#train1,val1 = train_test_split(train_xy, test_size = 0.25,random_state=42)#27811/13886

In [24]:
# print(train_xy.shape)
print(train.shape)
# print(val.shape)

(5620, 1392)


## 模型：

In [25]:
start_time = time.time()
offline=0
online=0
params={'booster':'gbtree',
    'objective': 'rank:pairwise',
    'eval_metric':'auc',
    'gamma':0.1,
    'min_child_weight':1.1,
    'max_depth':7,
    'lambda':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'colsample_bylevel':0.7,
    'eta': 0.01,
    'tree_method':'exact',
    'seed':1000,
    'nthread':12
    }

params1={
'booster':'gbtree',
'objective': 'binary:logistic',
'scale_pos_weight': 1/7.5,
#7183正样本
#55596条总样本
#差不多1:7.7这样子
'gamma':0.2,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':8, # 构建树的深度，越大越容易过拟合
'lambda':3,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本，测试集和训练集的百分比
#'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.03, # 如同学习率
'seed':1000,
'nthread':12,# cpu 线程数
'eval_metric': 'auc'
}

plst = list(params.items())
num_rounds = 5000 # 迭代次数

y = train.标签
X = train.drop(['标签','用户标识'],axis=1)
#X=train[feature_list]

val_y = val.标签
val_X = val.drop(['标签','用户标识'],axis=1)
#val_X = val[feature_list]

offline_test_X=offline_test.drop(['标签','用户标识'],axis=1)
online_test_X=online_test.drop(['标签','用户标识'],axis=1)
#offline_test_X=offline_test[feature_list]
#online_test_X=online_test[feature_list]

xgb_train = xgb.DMatrix(X, label=y)
xgb_val = xgb.DMatrix(val_X,label=val_y)

xgb_offline_test = xgb.DMatrix(offline_test_X)
xgb_online_test = xgb.DMatrix(online_test_X)

# return 训练和验证的错误率
watchlist = [(xgb_train, 'train'),(xgb_val, 'val')]

print ("跑到这里了xgb.train")
# training model 
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train,num_boost_round=7000,evals=watchlist,early_stopping_rounds=500)
print ("跑到这里了save_model")
model.save_model('D:/Files/QQFiles/935257580/FileRecv/personal loan/model/20170204_D_最好成绩特征训练集删去部分时间未知样本.model') # 用于存储训练出的模型
print ("best best_ntree_limit",model.best_ntree_limit)   #did not save the best,why?
print ("best best_iteration",model.best_iteration) #get it?

print ("跑到这里了model.predict")
preds_offline = model.predict(xgb_offline_test,ntree_limit=model.best_iteration)#
preds_online = model.predict(xgb_online_test,ntree_limit=model.best_iteration)#
offline=offline_test[['用户标识','标签']]
online=online_test[['用户标识']]

offline['预测']=preds_offline
online['预测']=preds_online

print("线下得分;")
print(ks(offline.预测,offline.标签))

print ("跑到这里了,输出结果")
from sklearn.preprocessing import MinMaxScaler
online.预测 = MinMaxScaler().fit_transform(online.预测)
output=online[['用户标识','预测']].rename(index=str, columns={"用户标识": "userid", "预测": "probability"})
output.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/output/20170204_D_最好成绩特征训练集删去部分时间未知样本.csv",index=None,encoding='utf-8')

#save feature score and feature information:  feature,score,min,max,n_null,n_gt1w
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('D:/Files/QQFiles/935257580/FileRecv/personal loan/feature_score/20170204_D_最好成绩特征训练集删去部分时间未知样本.csv','w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)
        
cost_time = time.time()-start_time
print ("",'\n',"cost time:",cost_time,"(s)")
print ("跑到这里了,结束！")
# CV训练
#xgb.cv(xgb_config, dtrain, num_boost_round=10, nfold=5, maximize=False, verbose_eval=True)

跑到这里了xgb.train
[0]	train-auc:0.829498	val-auc:0.724519
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 500 rounds.
[1]	train-auc:0.849125	val-auc:0.758536
[2]	train-auc:0.858206	val-auc:0.763464
[3]	train-auc:0.864993	val-auc:0.763277
[4]	train-auc:0.867714	val-auc:0.762845
[5]	train-auc:0.874225	val-auc:0.765074
[6]	train-auc:0.876946	val-auc:0.765555
[7]	train-auc:0.878419	val-auc:0.76395
[8]	train-auc:0.879983	val-auc:0.762278
[9]	train-auc:0.8819	val-auc:0.763907
[10]	train-auc:0.883689	val-auc:0.766753
[11]	train-auc:0.883541	val-auc:0.769082
[12]	train-auc:0.884774	val-auc:0.769824
[13]	train-auc:0.88555	val-auc:0.769695
[14]	train-auc:0.885459	val-auc:0.769561
[15]	train-auc:0.886342	val-auc:0.77605
[16]	train-auc:0.8872	val-auc:0.775807
[17]	train-auc:0.887586	val-auc:0.776273
[18]	train-auc:0.889244	val-auc:0.775992
[19]	train-auc:0.889791	val-auc:0.77574
[20]	train-auc:0.889883	val-auc:0.775364
[2

[196]	train-auc:0.920306	val-auc:0.7859
[197]	train-auc:0.920623	val-auc:0.786003
[198]	train-auc:0.920715	val-auc:0.786018
[199]	train-auc:0.920868	val-auc:0.786195
[200]	train-auc:0.921005	val-auc:0.785961
[201]	train-auc:0.921196	val-auc:0.786021
[202]	train-auc:0.921477	val-auc:0.7861
[203]	train-auc:0.921661	val-auc:0.785939
[204]	train-auc:0.921681	val-auc:0.785933
[205]	train-auc:0.921826	val-auc:0.785809
[206]	train-auc:0.921934	val-auc:0.785842
[207]	train-auc:0.922171	val-auc:0.785751
[208]	train-auc:0.922477	val-auc:0.78572
[209]	train-auc:0.922585	val-auc:0.785784
[210]	train-auc:0.92268	val-auc:0.785815
[211]	train-auc:0.922743	val-auc:0.785663
[212]	train-auc:0.923052	val-auc:0.785772
[213]	train-auc:0.92326	val-auc:0.785809
[214]	train-auc:0.923434	val-auc:0.785854
[215]	train-auc:0.923523	val-auc:0.785623
[216]	train-auc:0.923648	val-auc:0.785663
[217]	train-auc:0.923796	val-auc:0.785836
[218]	train-auc:0.923904	val-auc:0.78566
[219]	train-auc:0.924049	val-auc:0.785702


[393]	train-auc:0.946145	val-auc:0.789187
[394]	train-auc:0.94623	val-auc:0.789214
[395]	train-auc:0.94627	val-auc:0.789171
[396]	train-auc:0.946447	val-auc:0.789247
[397]	train-auc:0.946517	val-auc:0.789174
[398]	train-auc:0.946578	val-auc:0.789077
[399]	train-auc:0.946803	val-auc:0.789156
[400]	train-auc:0.946979	val-auc:0.789126
[401]	train-auc:0.947001	val-auc:0.789266
[402]	train-auc:0.947132	val-auc:0.789229
[403]	train-auc:0.947199	val-auc:0.789184
[404]	train-auc:0.947311	val-auc:0.789184
[405]	train-auc:0.947337	val-auc:0.789232
[406]	train-auc:0.947406	val-auc:0.789357
[407]	train-auc:0.947485	val-auc:0.789311
[408]	train-auc:0.947665	val-auc:0.789402
[409]	train-auc:0.947792	val-auc:0.789372
[410]	train-auc:0.947947	val-auc:0.789383
[411]	train-auc:0.948077	val-auc:0.789415
[412]	train-auc:0.948106	val-auc:0.789354
[413]	train-auc:0.948229	val-auc:0.789336
[414]	train-auc:0.948326	val-auc:0.789272
[415]	train-auc:0.948422	val-auc:0.78936
[416]	train-auc:0.948466	val-auc:0.78

[590]	train-auc:0.963704	val-auc:0.792051
[591]	train-auc:0.963777	val-auc:0.792142
[592]	train-auc:0.963818	val-auc:0.792075
[593]	train-auc:0.963846	val-auc:0.79203
[594]	train-auc:0.963948	val-auc:0.792084
[595]	train-auc:0.963984	val-auc:0.79213
[596]	train-auc:0.964043	val-auc:0.792078
[597]	train-auc:0.964167	val-auc:0.79209
[598]	train-auc:0.964254	val-auc:0.792133
[599]	train-auc:0.964318	val-auc:0.792008
[600]	train-auc:0.964454	val-auc:0.792048
[601]	train-auc:0.964517	val-auc:0.792011
[602]	train-auc:0.964586	val-auc:0.792051
[603]	train-auc:0.964665	val-auc:0.792172
[604]	train-auc:0.964782	val-auc:0.792185
[605]	train-auc:0.96484	val-auc:0.792203
[606]	train-auc:0.964915	val-auc:0.792194
[607]	train-auc:0.964989	val-auc:0.792197
[608]	train-auc:0.965112	val-auc:0.792102
[609]	train-auc:0.965318	val-auc:0.792096
[610]	train-auc:0.96547	val-auc:0.792197
[611]	train-auc:0.965531	val-auc:0.792142
[612]	train-auc:0.965596	val-auc:0.792084
[613]	train-auc:0.965644	val-auc:0.7920

KeyboardInterrupt: 

In [ ]:
# p1= pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/output/20170119_D_线下0.48901_线上0.43267.csv",encoding="gb2312") # 注意自己数据路径
# p2= pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/output/20170204_A_最好成绩复现.csv",encoding="gb2312") # 注意自己数据路径
# p=p1[['userid']]
# p['probability']=p1['probability']-p2['probability']